In [7]:
from datasets import load_dataset

squad = load_dataset("squad", split="train[:5000]")

Found cached dataset squad (C:/Users/tranq/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453)


In [8]:
squad = squad.train_test_split(test_size=0.2)

In [9]:
squad["train"][0]

{'id': '56cc92346d243a140015f039',
 'title': 'IPod',
 'context': "Video games are playable on various versions of iPods. The original iPod had the game Brick (originally invented by Apple's co-founder Steve Wozniak) included as an easter egg hidden feature; later firmware versions added it as a menu option. Later revisions of the iPod added three more games: Parachute, Solitaire, and Music Quiz.",
 'question': 'What was included on the first iPod?',
 'answers': {'text': ['Brick'], 'answer_start': [86]}}

In [10]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [11]:
def preprocess_function(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=384,
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        answer = answers[i]
        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label it (0, 0)
        if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

In [12]:
tokenized_squad = squad.map(preprocess_function, batched=True, remove_columns=squad["train"].column_names)

Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [13]:
from transformers import DefaultDataCollator

data_collator = DefaultDataCollator()

In [14]:
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer

model = AutoModelForQuestionAnswering.from_pretrained("distilbert-base-uncased")

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForQuestionAnswering: ['vocab_transform.bias', 'vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_projector.weight', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this mode

Train model

In [15]:
# training_args = TrainingArguments(
#     output_dir="my_awesome_qa_model",
#     evaluation_strategy="epoch",
#     learning_rate=2e-5,
#     per_device_train_batch_size=16,
#     per_device_eval_batch_size=16,
#     num_train_epochs=3,
#     weight_decay=0.01
# )

# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=tokenized_squad["train"],
#     eval_dataset=tokenized_squad["test"],
#     tokenizer=tokenizer,
#     data_collator=data_collator,
# )

# trainer.train()

In [32]:
question = "Bạn là ai?"
context = "Tôi là Quốc Tuấn"

In [33]:
from transformers import pipeline

question_answerer = pipeline("question-answering", model="C:/Users/tranq/Downloads/kltn/my_awesome_qa_model/checkpoint-500")
question_answerer(question=question, context=context)

{'score': 0.18669308722019196,
 'start': 0,
 'end': 16,
 'answer': 'Tôi là Quốc Tuấn'}

In [34]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("C:/Users/tranq/Downloads/kltn/my_awesome_qa_model/checkpoint-500")
inputs = tokenizer(question, context, return_tensors="pt")

In [35]:
from transformers import AutoModelForQuestionAnswering
import torch

model = AutoModelForQuestionAnswering.from_pretrained("C:/Users/tranq/Downloads/kltn/my_awesome_qa_model/checkpoint-500")
with torch.no_grad():
    outputs = model(**inputs)

In [36]:
answer_start_index = outputs.start_logits.argmax()
answer_end_index = outputs.end_logits.argmax()

In [37]:
predict_answer_tokens = inputs.input_ids[0, answer_start_index : answer_end_index + 1]
tokenizer.decode(predict_answer_tokens)

'ban la ai? [SEP] toi la quoc tuan'